# **AI Mental Health Companion (LLM)**

In [1]:
# Step 1: Install required libraries
!pip install transformers datasets torch huggingface_hub --quiet

In [2]:
# Step 2: Import libraries
import os
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from huggingface_hub import login

In [3]:
# Step 3: Hugging Face authentication

from google.colab import userdata
HF_TOKEN = userdata.get("HF_TOKEN")

if HF_TOKEN is None:
    raise ValueError("Hugging Face token not found")

print("Token loaded!")
login(HF_TOKEN)

Token loaded!


In [4]:
# Step 4: Load your CSV file
df = pd.read_csv("/content/drive/MyDrive/mental_health.csv")

In [5]:
# Step 5: Preprocess CSV to split HUMAN and ASSISTANT
df[['input_text', 'target_text']] = df['text'].str.extract(r'<HUMAN>:\s*(.*?)\s*<ASSISTANT>:\s*(.*)')
df = df.dropna()
dataset = Dataset.from_pandas(df[['input_text', 'target_text']])
print("Dataset:", dataset)

Dataset: Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 172
})


In [6]:
# Step 6: Load Llama-3.2-1B-Instruct
model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=HF_TOKEN)

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1010: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [7]:
# Step 7: Create chat pipeline
chat = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [8]:
# Step 8: Define system prompt
system_prompt = """
You are a warm, supportive mental health chatbot.
Respond with ONE short, clear, and practical suggestion tailored to the user's concern.
Avoid listing multiple options, repeating phrases, or using bullet points or numbers.
Keep your tone empathetic, conversational, and calming.
If the user shares progress, acknowledge it with encouragement.
"""

In [11]:
# Step 9: Interactive LOOP
import re

print("🤖 Welcome to Mental Health Chatbot (type 'exit' to quit)\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("🤖 Bot: Take care! Goodbye 💙")
        break

    prompt = f"{system_prompt}\nUser: {user_input}\nAssistant:"

    response = chat(
        prompt,
        max_new_tokens=80,
        do_sample=True,
        truncation=True,
        pad_token_id=tokenizer.eos_token_id,
        return_full_text=False)

    bot_reply = response[0]['generated_text']
    bot_reply = bot_reply.replace(prompt, '').strip()
    bot_reply = re.sub(r'(Assistant:)+', '', bot_reply).strip()

    print("🤖 Bot:", bot_reply)

🤖 Welcome to Mental Health Chatbot (type 'exit' to quit)

You: i am feeling stress today, please suggest one tip
🤖 Bot: Sometimes, taking a few deep breaths can help calm the mind and body. Try inhaling for a count of 4, holding for 7, and exhaling for 8. You can do this anywhere, even just a minute or two. It might feel silly, but it can be really helpful in managing stress.
You: yes i feel fresh and stress reduced
🤖 Bot: I'm so glad to hear that! It sounds like you're on the right track. Keep up the good work and remember to take care of yourself. What's one thing you can do this week to show yourself some extra love and self-care?
You: bye
🤖 Bot: Take care! Goodbye 💙


In [10]:
# Step 10: Test chat with a sample input
sample_input = "I am feeling stress today. What should I do?"

prompt = f"{system_prompt}\nUser: {sample_input}\nAssistant:"

# Generate response
response = chat(
    prompt,
    max_new_tokens=80,
    do_sample=True,
    truncation=True,
    pad_token_id=tokenizer.eos_token_id
)

bot_reply = response[0]['generated_text']
bot_reply = re.sub(r'\d+\.\s*', '', bot_reply)
bot_reply = re.sub(r'[-*]\s*', '', bot_reply)
bot_reply = bot_reply.replace(prompt, '').strip()

print("🤖 Bot response:", bot_reply)


🤖 Bot response: Sometimes, taking a short walk outside can help clear your mind. Why don't you take 1015 minutes to step outside and breathe in some fresh air? You might find it helps calm you down.
